In this test file, we will use PySCF package to calculate the energy other than Gaussian software.

The purpose of this test is to check the availability of using PySCF package in cmaes_optimizer.

In [1]:
# import necessary packages

import cma
from pyscf import gto, dft, lib

In [2]:
# basic setting of calculations

dft_keyword = "B3LYP"    # the keyword of DFT to calculate the energy
basis_keyword = "6-31G*"    # the basis set to calculate the energy
charge = 0    # total charge of system
multiplicity = 1    # multiplicity of system
max_memory = 4000    # maximum memory allowed in DFT calculation, in MB
cal_method = "R"    # set the calculation method to be "R" (restricted), "U" (unrestricted), "RO" (restricted open-shell)
#lib.misc.num_threads(n = 4)    # PySCF uses all CPU cores to do calculations as default; users may change it if needed

In [3]:
# calculate the energy

def energy_calculate(atom_list, dft_keyword, basis_keyword, charge=0, multiplicity=1, cal_method="R"):
    if cal_method != "RO" and multiplicity != 1:    # set default calculation method; R for close-shell systems; U for open-shell systems
        cal_method = "U"
    try:
        mol = gto.Mole()    # set the molecule

        coord = ""    # translate the coordinates
        for atom in atom_list:
            coord += atom
            coord += "; "
        mol.atom = coord

        mol.charge = charge
        mol.multiplicity = multiplicity
        mol.basis = basis_keyword
        mol.max_memory = max_memory
        
        if cal_method == "R":
            scf = dft.RKS(mol.build())    # set the R DFT calculation
        elif cal_method == "U":
            scf = dft.UKS(mol.build())    # set the U DFT calculation
        elif cal_method == "RO":
            scf = dft.ROKS(mol.build())    # set the U DFT calculation

        scf.verbose = 2    # use simple output model
        scf.max_cycle = 128    # maximum cycle number of SCF iterations
        scf.conv_tol = 1e-8    # converge standard of SCF iteration
        scf.grids.atom_grid = (75, 302)    # set the integration grid, (75, 302) is the default level (int=fine) in Gaussian 09
        scf.xc = dft_keyword
        
        energy = scf.kernel()    # calculate the energy

        if scf.converged == False:    # if SCF iterations failed, return a very high energy
            energy == 999.999

        return energy

    except:
        return 999.999    # if energy calculation failed, return a very high energy

In [4]:
# define energy function for a specific system

def energy(coord):
    atom_list = ["O", "H 1 "+str(coord[0]), "H 1 "+str(coord[0])+" 2 "+str(coord[1])]    # use z-martix as an example and keep two O-H bonds the same
    return energy_calculate(atom_list, dft_keyword, basis_keyword, charge, multiplicity)

In [5]:
# cma-es part

init_coord = [1.0, 100.0]    # initial coordinates of atoms; this time we use the initial coordinates not that near the result

es = cma.CMAEvolutionStrategy(init_coord, 0.25, {"seed":2445})    # set cma-es model, use 0.25 as sigma0, 2445 as random seed

es.optimize(energy)    # optimize the cma-es model

(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=2445, Thu Oct 13 15:01:56 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 -7.636950990464416e+01 1.0e+00 2.07e-01  2e-01  2e-01 0:01.8
    2     12 -7.636928060814114e+01 1.1e+00 2.19e-01  2e-01  2e-01 0:03.6
    3     18 -7.636313445628745e+01 1.5e+00 2.13e-01  1e-01  2e-01 0:05.5
    5     30 -7.636938614564643e+01 2.2e+00 1.58e-01  7e-02  2e-01 0:09.0
    8     48 -7.636958064149479e+01 3.3e+00 9.67e-02  3e-02  8e-02 0:13.9
   11     66 -7.636947658516142e+01 3.7e+00 7.20e-02  1e-02  6e-02 0:19.0
   15     90 -7.636955982818422e+01 8.0e+00 5.02e-02  5e-03  4e-02 0:25.5
   20    120 -7.636957490209609e+01 1.2e+01 3.72e-02  2e-03  3e-02 0:33.8
   25    150 -7.636959326188970e+01 2.4e+01 6.12e-02  2e-03  5e-02 0:42.1
   31    186 -7.636964585614504e+01 5.3e+01 1.25e-01  2e-03  1e-01 0:52.0
   37    222 -7.636972822408075e+01 9.7e+01 1.70e-01  2e-03  2e-01 1:02.1
   44    264 -7.6369854505

In [6]:
print(es.result.xbest)    # print optimization result

[  0.96891999 103.85329792]


The optimization result is:

O-H bond is 0.96891999 A;

H-O-H angle is 103.85329792 degree.

****************

If we directly use Gaussian to do optimization with "opt=tight", we will get the result:

O-H bond is 0.9687 A;

H-O-H angle is 103.6522 degree.

****************

The result is close, the small difference might be caused from the difference of PySCF and Gaussian.

It is possible to use CMA-ES algorithm to use PySCF package in cmaes_optimizer.